# Ranking Utils

Contains the ratings_to_rankings function and related evaluation metrics

In [ ]:
import numpy as np
from typing import List
from scipy.stats import kendalltau, spearmanr

In [ ]:
def ratings_to_rankings(ratings: np.ndarray) -> List[np.ndarray]:
    """
    Converts a 2D array of ratings into rankings for each row.

    Each row in the input represents ratings for a set of items. The function returns
    the indices of the items ranked in descending order of their rating for each row.

    Parameters:
        ratings (np.ndarray): A 2D NumPy array of shape (n_samples, n_items), where each row contains ratings for items.

    Returns:
        List[np.ndarray]: A list of 1D NumPy arrays. Each array contains the indices that would sort the corresponding
                          row of ratings in descending order (highest rated item first).

    Example:
        >>> ratings = np.array([[3.2, 5.1, 1.0], [2.0, 4.0, 3.5]])
        >>> ratings_to_rankings(ratings)
        [array([1, 0, 2]), array([1, 2, 0])]
    """
    return [np.argsort(row)[::-1] for row in ratings]

In [ ]:
def compute_ranking_metrics(targets: List, preds: List) -> dict[str, float | List]:

    results = {}

    results["overall_accuracy"] = _exact_match_accuracy(targets, preds)
    results["all_kendall_tau_correlations"] = _kendall_tau_correlation(targets, preds)
    results["all_kendall_tau_distances"] = _kendall_tau_distance(targets, preds)
    results["all_spearman_footrule_distances"] = _spearman_footrule_distance(
        targets, preds
    )
    results["all_spearman_rho_correlations"] = _spearman_rank_correlation(
        targets, preds
    )

    return results


def _exact_match_accuracy(target: List, pred: List) -> float:
    total_comparisons = len(target)
    mismatches_count = 0

    for i in range(total_comparisons):
        target_indices = (
            np.array(target[i]) if not isinstance(target[i], np.ndarray) else target[i]
        )
        pred_indices = (
            np.array(pred[i]) if not isinstance(pred[i], np.ndarray) else pred[i]
        )

        if not np.array_equal(target_indices, pred_indices):
            mismatches_count += 1

    overall_accuracy = (
        (total_comparisons - mismatches_count) / total_comparisons
    ) * 100

    return overall_accuracy


def _kendall_tau_correlation(target: List, pred: List) -> List:
    total_comparisons = len(target)
    all_correlations = []

    for i in range(total_comparisons):
        target_ranks = (
            np.array(target[i]) if not isinstance(target[i], np.ndarray) else target[i]
        )
        model_ranks = (
            np.array(pred[i]) if not isinstance(pred[i], np.ndarray) else pred[i]
        )

        # Kendall Tau Distance
        tau_correlation, _ = kendalltau(target_ranks, model_ranks)
        all_correlations.append(tau_correlation)

    return all_correlations


def _kendall_tau_distance(target: List, pred: List) -> List:

    total_comparisons = len(target)
    n = len(target[0])
    max_disagreements = n * (n - 1) / 2  # number of all possible pairs

    all_distances = []

    for i in range(total_comparisons):
        # Kendall Tau Distance
        tau_correlation, _ = kendalltau(target[i], pred[i])
        assert isinstance(tau_correlation, np.float64)

        kendall_tau_distance = max_disagreements * (1 - tau_correlation) / 2
        all_distances.append(kendall_tau_distance)

    return all_distances


def _spearman_footrule_distance(target: List, pred: List) -> List:
    total_comparisons = len(target)
    all_spearman_footrules = []

    for i in range(total_comparisons):
        target_ranks = (
            np.array(target[i]) if not isinstance(target[i], np.ndarray) else target[i]
        )
        model_ranks = (
            np.array(pred[i]) if not isinstance(pred[i], np.ndarray) else pred[i]
        )

        # Spearman Footrule Distance
        # Sum of absolute differences between positions
        footrule_distance = np.sum(np.abs(target_ranks - model_ranks))
        all_spearman_footrules.append(footrule_distance)

    return all_spearman_footrules


def _spearman_rank_correlation(target: List, pred: List) -> List:
    total_comparisons = len(target)
    all_spearman_rhos = []

    for i in range(total_comparisons):
        target_ranks = (
            np.array(target[i]) if not isinstance(target[i], np.ndarray) else target[i]
        )
        model_ranks = (
            np.array(pred[i]) if not isinstance(pred[i], np.ndarray) else pred[i]
        )

        # Spearman Rank Correlation
        spearman_rho, _ = spearmanr(target_ranks, model_ranks)
        all_spearman_rhos.append(spearman_rho)

    return all_spearman_rhos
